In [2]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_community.graphs import Neo4jGraph
# from neo4j_graphrag import Text2Graph
from langchain_neo4j import Neo4jVector
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_neo4j import Neo4jGraph
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from langchain_neo4j import GraphCypherQAChain
# from langchain.graphs.neo4j_graph import Neo4jGraph
from dotenv import load_dotenv

In [3]:
from dotenv import load_dotenv

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

In [5]:
prompt = ChatPromptTemplate.from_template(template)

In [31]:
model = OllamaLLM(model="mistral-nemo:latest")

chain = prompt | model

In [32]:
os.getenv("URL") == 'neo4j+s://def446e2.databases.neo4j.io'

True

In [33]:
os.getenv("USERNAME")

'neo4j'

In [34]:
os.getenv("PASSWORD") == 'rFUw7eNgKtAwqXkU1NrUW4I0NfNa8zdrVFUkyV8h8FE'

True

In [35]:
# from neo4j import GraphDatabase
graph = Neo4jGraph(
    url=os.getenv("URL"),
    username=os.getenv("USERNAME"),
    password=os.getenv("PASSWORD")
)

In [6]:
cypher_build = """
// --- Core Person Node ---
MERGE (einstein:Person {name: 'Albert Einstein', birth_date: '14 March 1879', death_date: '18 April 1955'})
SET einstein.nationality = 'German-born', einstein.profession = 'Theoretical Physicist'

// --- Theories & Contributions ---
MERGE (relativity:Theory {name: 'Theory of Relativity'})
MERGE (quantum_mechanics:Field {name: 'Quantum Mechanics'})
MERGE (mass_energy:Concept {name: 'Mass–Energy Equivalence', formula: 'E = mc^2'})
MERGE (photoelectric:Law {name: 'Law of the Photoelectric Effect'})
MERGE (general_relativity:Theory {name: 'General Theory of Relativity'})
MERGE (special_relativity:Theory {name: 'Special Theory of Relativity'})
MERGE (brownian_motion:Phenomenon {name: 'Brownian Motion'})
MERGE (bose:Person {name: 'Satyendra Nath Bose'})
MERGE (bose_einstein_stats:Concept {name: 'Bose–Einstein Statistics'})

// --- Institutions & Awards ---
MERGE (nobel:Award {name: 'Nobel Prize in Physics', year: 1921})
MERGE (univ_zurich:Institution {name: 'University of Zurich'})
MERGE (polytechnic:Institution {name: 'Swiss Federal Polytechnic School in Zurich'})
MERGE (prussian_academy:Institution {name: 'Prussian Academy of Sciences'})
MERGE (humboldt:Institution {name: 'Humboldt University of Berlin'})
MERGE (kaiser:Institution {name: 'Kaiser Wilhelm Institute for Physics'})
MERGE (swiss_patent:Institution {name: 'Swiss Patent Office in Bern'})
MERGE (usa:Country {name: 'United States'})
MERGE (germany:Country {name: 'Germany'})
MERGE (switzerland:Country {name: 'Switzerland'})

// --- Key Relationships ---
MERGE (einstein)-[:DEVELOPED]->(relativity)
MERGE (einstein)-[:CONTRIBUTED_TO]->(quantum_mechanics)
MERGE (einstein)-[:FORMULATED]->(mass_energy)
MERGE (einstein)-[:DISCOVERED]->(photoelectric)
MERGE (einstein)-[:PROPOSED]->(general_relativity)
MERGE (einstein)-[:PUBLISHED]->(brownian_motion)
MERGE (einstein)-[:CO_WORKED_WITH]->(bose)
MERGE (einstein)-[:CO_CREATED]->(bose_einstein_stats)

MERGE (einstein)-[:AWARDED]->(nobel)
MERGE (nobel)-[:FOR_DISCOVERY_OF]->(photoelectric)

MERGE (einstein)-[:STUDIED_AT]->(polytechnic)
MERGE (einstein)-[:EARNED_PHD_FROM]->(univ_zurich)
MERGE (einstein)-[:WORKED_AT]->(swiss_patent)
MERGE (einstein)-[:JOINED]->(prussian_academy)
MERGE (einstein)-[:JOINED]->(humboldt)
MERGE (einstein)-[:DIRECTED]->(kaiser)

MERGE (einstein)-[:BORN_IN]->(germany)
MERGE (einstein)-[:MOVED_TO]->(switzerland)
MERGE (einstein)-[:BECAME_CITIZEN_OF]->(switzerland)
MERGE (einstein)-[:MOVED_TO]->(usa)
MERGE (einstein)-[:BECAME_CITIZEN_OF]->(usa)

MERGE (einstein)-[:OPPOSED]->(:Concept {name: 'Quantum Randomness'})
MERGE (einstein)-[:WORKED_ON]->(:Theory {name: 'Unified Field Theory'})
MERGE (einstein)-[:SAID]->(:Quote {text: 'God does not play dice'})

// --- Historical Event Connection ---
MERGE (hitler:Person {name: 'Adolf Hitler'})
MERGE (nazi_persecution:Event {name: 'Nazi Persecution of Jews'})
MERGE (einstein)-[:FLED_FROM]->(nazi_persecution)
MERGE (nazi_persecution)-[:LED_BY]->(hitler)

MERGE (roosevelt:Person {name: 'Franklin D. Roosevelt'})
MERGE (nuclear_letter:Event {name: 'Einstein–Szilard Letter'})
MERGE (einstein)-[:ENDORSED]->(nuclear_letter)
MERGE (nuclear_letter)-[:ADDRESSED_TO]->(roosevelt)
MERGE (nuclear_letter)-[:ABOUT]->(:Topic {name: 'German Nuclear Weapons Program'})

"""

In [7]:
summary = driver.execute_query(cypher_build, database_="neo4j").summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

/tmp/ipykernel_17566/3887568696.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary = driver.execute_query(cypher_build, database_="neo4j").summary


Created 28 nodes in 1782 ms.


In [36]:
chain_cypher = GraphCypherQAChain.from_llm(llm=model, graph=graph, verbose=True,allow_dangerous_requests = True)

## Asking questions based on the knowledge graph.

In [37]:
question = "Where was Aum Working"
result = chain_cypher.invoke({"query": question})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Aum'})-[:WORKS_FOR|WORKED_AT]->(c:Company) RETURN c
Full Context:
[{'c': {'name': 'moon finance'}}]

> Finished chain.


In [39]:
print(result["result"])

I don't know the answer.


## Inserting new memories :

In [18]:
new_memory = "Albert Einstien had a girlfriend named Sofia"
documents = [Document(page_content=new_memory)]

In [24]:
graph_transformer = LLMGraphTransformer(llm=model)
# graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)
# graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [27]:
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

AttributeError: 'list' object has no attribute 'get'

In [26]:
graph_documents[0].nodes  # list of nodes
graph_documents[0].relationships  # list of relationships

TypeError: 'coroutine' object is not subscriptable

In [34]:
await update_graph_in_neo4j(graph_documents)

In [35]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Sofia', type='Person', properties={}), Node(id='Albert Einstien', type='Person', properties={})]
Relationships:[Relationship(source=Node(id='Albert Einstien', type='Person', properties={}), target=Node(id='Sofia', type='Person', properties={}), type='HAS_GIRLFRIEND', properties={})]


In [ ]:
# Aiyo KG hatt naa jaye puraka pura:
summary = driver.execute_query(cypher_query_manual, database_="neo4j").summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))